# Binance Class API

In [33]:
import asyncio
import websockets
import json
import datetime
import requests
import hashlib
import urllib.parse
import hmac
import time
import logging
import os
from collections import deque
import numpy as np

class BinanceAPI:
    def __init__(self, api_key: str, api_secret: str, testnet: bool = False):
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_url = "https://testnet.binancefuture.com" if testnet else "https://fapi.binance.com"

    def _generate_signature(self, params: dict) -> str:
        """Generate HMAC SHA256 signature for request."""
        query = urllib.parse.urlencode(params)
        return hmac.new(self.api_secret.encode(), query.encode(), hashlib.sha256).hexdigest()

    def _send_request(self, method: str, endpoint: str, params=None, signed=False) -> dict:
        """Send request to Binance API."""
        url = f"{self.base_url}{endpoint}"
        headers = {"X-MBX-APIKEY": self.api_key}
        if params is None:
            params = {}
        if signed:
            # Add timestamp in milliseconds
            params["timestamp"] = int(time.time() * 1000)
            print(f"Generated timestamp: {params['timestamp']}")  # Log the timestamp
            # Generate signature
            params["signature"] = self._generate_signature(params)
        print(f"Sending {method} request to {url} with params: {params}")  # Log the full request
        try:
            if method == "GET":
                response = requests.get(url, headers=headers, params=params)
            elif method == "POST":
                # Send params as form data for POST requests
                response = requests.post(url, headers=headers, data=params)
            elif method == "DELETE":
                response = requests.delete(url, headers=headers, params=params)
            else:
                raise ValueError(f"Invalid method: {method}")
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            # Capture the complete error response
            error_message = str(e)
            if hasattr(e, 'response') and e.response is not None:
                try:
                    error_message = e.response.json()  # Try to parse the JSON error response
                except ValueError:
                    error_message = e.response.text  # Fallback to raw text if JSON parsing fails
            print(f"API request failed: {error_message}")
            return {"error": error_message}  # Return the error message in a structured way

    def get_mark_price(self, symbol: str) -> dict:
        """Get mark price for a symbol."""
        return self._send_request("GET", "/fapi/v1/premiumIndex", {"symbol": symbol})

    def get_position_risk(self, symbol: str = None) -> dict:
        """Get position risk information."""
        params = {"symbol": symbol} if symbol else {}
        return self._send_request("GET", "/fapi/v3/positionRisk", params, signed=True)
    
    def get_symbol_info(self, symbol: str) -> dict:
        """Fetch symbol information (e.g., tick size, lot size)."""
        endpoint = "/fapi/v1/exchangeInfo"
        response = self._send_request("GET", endpoint)
        if response and "symbols" in response:
            for s in response["symbols"]:
                if s["symbol"] == symbol:
                    return s
        return None
    
    def get_position_size(self, symbol: str) -> float:
        """Fetch the current position size for a symbol."""
        positions = self.get_position_risk(symbol=symbol)
        if isinstance(positions, list) and len(positions) > 0:
            position = positions[0]
            return float(position.get('positionAmt', 0))
        return 0

    def create_order(self, symbol: str, side: str, order_type: str, quantity: float, 
                    price: float = None, time_in_force: str = None, 
                    reduce_only: bool = False, stopPrice: float = None) -> dict:
        """Create a new order on Binance Futures."""
        params = {
            "symbol": symbol,
            "side": side,
            "type": order_type,
            "quantity": quantity,
            "reduceOnly": reduce_only,
        }

        # Handle STOP_MARKET orders
        if order_type == "STOP_MARKET":
            if stopPrice is None:
                raise ValueError("stopPrice is required for STOP_MARKET orders.")
            params["stopPrice"] = stopPrice  # Ensure correct spelling

        # Handle LIMIT orders
        if order_type == "LIMIT":
            if price is None or time_in_force is None:
                raise ValueError("Price and timeInForce are required for LIMIT orders.")
            params["price"] = price
            params["timeInForce"] = time_in_force

        return self._send_request("POST", "/fapi/v1/order", params, signed=True)


    def cancel_all_orders(self, symbol: str) -> dict:
        """Cancel all open orders for a symbol."""
        endpoint = "/fapi/v1/allOpenOrders"
        params = {"symbol": symbol}
        return self._send_request("DELETE", endpoint, params, signed=True)
    
    def close_position(self, symbol: str):
        """Close the active position for a specific symbol."""
        positions = self.get_position_risk()
        if positions is None:
            print("Failed to retrieve position information.")
            return

        for position in positions:
            if position['symbol'] == symbol:
                position_amt = float(position['positionAmt'])

                if position_amt != 0:
                    side = 'SELL' if position_amt > 0 else 'BUY'
                    quantity = abs(position_amt)

                    response = self.create_order(
                        symbol=symbol,
                        side=side,
                        order_type="MARKET",
                        quantity=quantity,
                        reduce_only=True
                    )

                    if response:
                        print(f"Closed position: {symbol} with quantity {quantity}")
                    else:
                        print(f"Failed to close position: {symbol}")
                else:
                    print(f"No active position found for {symbol}.")
                return

        print(f"No position information found for {symbol}.")

In [34]:
# Initialize Binance API
binance_api = BinanceAPI(api_key="StZt5sP9jn4GIaBRohoAWJJ9y1QJIILOIFgjD5TuctFprI2DS2MRMX8zuBrpdWkm", 
                         api_secret="0hlhVwVcDvhlPwF8SbXXkAyWe8b7yFIwI23VdIiNJGDLpBJWugCgHgZ61FSk7Set")

In [39]:
def place_order(side, symbol, quantity=40):
            """Place a new market order."""
            try:
                response = binance_api.create_order(
                    symbol=symbol,
                    side=side,
                    order_type="MARKET",
                    quantity=quantity
                )

                # Check if response contains an error
                if response is None or "error" in response:
                    error_msg = response.get("error", "No response from Binance API")
                    print(f"Order execution failed: {error_msg}")
                    return None

                # Log the order execution
                if "orderId" in response:
                    print(f"{side} order executed: {response}")
                    return response
                else:
                    print(f"Order failed: {response}")
                    return None
            except Exception as e:
                print(f"Order execution error: {str(e)}")
                return None


def place_tp(side, symbol, price, tp_percent):
    """Place or update take-profit order."""
    try:
        if tp_percent is None:
            return

        # Calculate TP price
        tp_price = round(price * (1 + tp_percent / 100), 5)
        
        # Determine order side based on position direction
        order_side = "SELL" if side == "BUY" else "BUY"
        
        tp_response = binance_api.create_order(
            symbol=symbol,
            side=order_side,
            order_type="LIMIT",
            quantity=40,
            price=tp_price,
            time_in_force="GTC",
            reduce_only=True
        )
        
        if "orderId" in tp_response:
            print(f"📈 Take-profit order placed at {tp_price:.5f}: {tp_response}")
        else:
            print(f"❌ Take-profit order failed: {tp_response}")
            
    except Exception as e:
        print(f"🔥 TP placement error: {str(e)}")

import time

def place_sl_market(side, symbol, price, quantity, sl_percent):
    """Place a stop-loss market order."""
    try:
        if sl_percent is None:
            return

        # Calculate Stop-Loss price
        sl_price = price * (1 - (sl_percent + 0.02) / 100)

        # Determine order side
        order_side = "SELL" if side == "BUY" else "BUY"

        # Adjust decimal precision based on symbol
        decimal_precision = 5  # Default for most altcoins
        if symbol.endswith("USDT"):
            decimal_precision = 2 if symbol.startswith("BTC") else 5  # BTC: 2 decimals, others: 5

        # Place stop-loss market order
        sl_response = binance_api.create_order(
            symbol=symbol,
            side=order_side,
            order_type="STOP_MARKET",
            quantity=quantity,
            reduce_only=True,
            stopPrice=round(sl_price, decimal_precision)  # Dynamically adjust precision
        )

        if sl_response and "orderId" in sl_response:
            print(f"📉 Stop-loss market order placed at {round(sl_price, decimal_precision)}: {sl_response}")
        else:
            print(f"❌ Stop-loss market order failed: {sl_response}")

    except Exception as e:
        print(f"🔥 SL Market placement error: {str(e)}")


def place_sl_post_only(side, symbol, price, quantity, sl_percent, sl_offset=0.0001):
    """Place a Stop-Limit (Post-Only) order for Binance Options."""
    try:
        if sl_percent is None:
            return

        # Calculate Stop Price and Limit Price
        stop_price = price * (1 - sl_percent / 100)
        limit_price = stop_price - sl_offset  # Slightly below stop price to prevent immediate execution

        # Determine order side based on position direction
        order_side = "SELL" if side == "BUY" else "BUY"

        params = {
            "symbol": symbol,
            "side": order_side,
            "type": "LIMIT",  # Only limit orders are supported
            "quantity": quantity,
            "price": round(limit_price, 5),  # Ensure proper price precision
            "timeInForce": "GTC",  # Default is Good-Til-Cancelled
            "reduceOnly": True,  # Ensures it only reduces the position
            "postOnly": True,  # Enforces Post-Only behavior
            "newOrderRespType": "RESULT",  # Get full response
            "recvWindow": 5000,  # 5-second timeout window
            "timestamp": int(time.time() * 1000)
        }

        sl_response = binance_api.create_order(**params)  # Make async request

        if sl_response and "orderId" in sl_response:
            print(f"✅ Post-Only Stop-Limit placed at {stop_price:.5f}, Limit: {limit_price:.5f}")
        else:
            print(f"❌ Stop-Limit Post-Only failed: {sl_response}")

    except Exception as e:
        print(f"🔥 SL Post-Only placement error: {str(e)}")

# Test

In [11]:
symbol = "DOGEUSDT"
tp_percent = 1
sl_percent = 0.05

In [21]:
place_order = place_order("BUY", symbol)
print(place_order)

Generated timestamp: 1742885914270
Sending POST request to https://fapi.binance.com/fapi/v1/order with params: {'symbol': 'DOGEUSDT', 'side': 'BUY', 'type': 'MARKET', 'quantity': 40, 'reduceOnly': False, 'timestamp': 1742885914270, 'signature': '7b53d0cc7602eb154cc003de321c2c85bd4f6f776d090abf5f3f124b353b8f87'}
BUY order executed: {'orderId': 70127570747, 'symbol': 'DOGEUSDT', 'status': 'NEW', 'clientOrderId': 'ibcJWhIGrRNpIW7J2RlnIJ', 'price': '0.000000', 'avgPrice': '0.00', 'origQty': '40', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.000000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.000000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1742885914378}
{'orderId': 70127570747, 'symbol': 'DOGEUSDT', 'status': 'NEW', 'clientOrderId': 'ibcJWhIGrRNpIW7J2RlnI

In [13]:
mark_price_response = binance_api.get_mark_price(symbol=symbol)
mark_price = float(mark_price_response["markPrice"])

Sending GET request to https://fapi.binance.com/fapi/v1/premiumIndex with params: {'symbol': 'DOGEUSDT'}


In [14]:
place_tp("BUY", symbol, mark_price, tp_percent)

Generated timestamp: 1742885691073
Sending POST request to https://fapi.binance.com/fapi/v1/order with params: {'symbol': 'DOGEUSDT', 'side': 'SELL', 'type': 'LIMIT', 'quantity': 40, 'reduceOnly': True, 'price': 0.18503, 'timeInForce': 'GTC', 'timestamp': 1742885691073, 'signature': '0d2991defb5d47bc6c0f7d11609196843f44185c30f8503073c79c5e673167f8'}
📈 Take-profit order placed at 0.18503: {'orderId': 70127468079, 'symbol': 'DOGEUSDT', 'status': 'NEW', 'clientOrderId': 'LMrbLnm49NJQPbIEdsko49', 'price': '0.185030', 'avgPrice': '0.00', 'origQty': '40', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.000000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': True, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.000000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1742885691134}


In [23]:
place_sl_post_only("BUY", symbol, mark_price, 40, sl_percent)

🔥 SL Post-Only placement error: BinanceAPI.create_order() got an unexpected keyword argument 'type'


In [41]:
place_sl_market("BUY", symbol, mark_price, 40, sl_percent)

Generated timestamp: 1742886298910
Sending POST request to https://fapi.binance.com/fapi/v1/order with params: {'symbol': 'DOGEUSDT', 'side': 'SELL', 'type': 'STOP_MARKET', 'quantity': 40, 'reduceOnly': True, 'stopPrice': 0.18307, 'timestamp': 1742886298910, 'signature': 'd26a68b1bfd809ce143d016be6a7c3555edc41c89118acea1828a62b77810a1b'}
📉 Stop-loss market order placed at 0.18307: {'orderId': 70127815971, 'symbol': 'DOGEUSDT', 'status': 'NEW', 'clientOrderId': '70YPm0PTccWLocBbU9Gj5R', 'price': '0.000000', 'avgPrice': '0.00', 'origQty': '40', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.000000', 'timeInForce': 'GTC', 'type': 'STOP_MARKET', 'reduceOnly': True, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.183070', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'STOP_MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1742886298966}
